In [ ]:
# Python imports

import itertools
import lightgbm as lgb
import numpy as np
import pandas as pd
import re
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, log_loss
from sklearn.model_selection import KFold

# Custom functions

def id_cat_feat(df):
    return [col for col in df.columns if 2 < df[col].nunique() < 10 and "days" not in col]

def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def permutation_test(y_true, model1_scores, model2_scores, n_permutations=1000):
    observed_diff = average_precision_score(y_true, model1_scores) - average_precision_score(y_true, model2_scores)
    n_model1 = len(model1_scores)
    n_model2 = len(model2_scores)
    all_scores = np.concatenate([model1_scores, model2_scores])
    shuffled_diffs = np.empty(n_permutations)

    for i in range(n_permutations):
        np.random.shuffle(all_scores)
        shuffled_model1_scores = all_scores[:n_model1]
        shuffled_model2_scores = all_scores[n_model1:]
        shuffled_diffs[i] = average_precision_score(y_true, shuffled_model1_scores) - average_precision_score(y_true, shuffled_model2_scores)

    p_value = np.mean(np.abs(shuffled_diffs) >= np.abs(observed_diff))
    return p_value

def bootstrap(dataframe, predictor_col, outcome_col, metric_function, n_bootstrap=500, resample_frac=1, confidence_level=0.95):
    sample_size = int(len(dataframe) * resample_frac)
    metric_values = []

    for _ in range(n_bootstrap):
        sample = dataframe.sample(n=sample_size, replace=True)
        metric_value = metric_function(sample[outcome_col], sample[predictor_col])
        metric_values.append(metric_value)

    metric_values = np.array(metric_values)
    mean = np.mean(metric_values)
    alpha = 1 - confidence_level
    lower_bound = np.percentile(metric_values, 100 * alpha / 2)
    upper_bound = np.percentile(metric_values, 100 * (1 - alpha / 2))

    return mean, lower_bound, upper_bound

## Hyperparameter tuning

In [ ]:
for dx in ['afib','cad','celiac','gallstone','polyp','t2d','varicose','vte']:
    
    dx_name = dx+'_status'

    cd = pd.read_pickle('./Final/cases.pkl')
    indata = pd.read_pickle('./Final/indata.pkl')

    indata = indata.merge(cd[['f.eid',dx_name]], on='f.eid', how='inner').drop_duplicates('f.eid')
    indata = indata.loc[indata[dx_name] != -1]
    indata.loc[indata[dx_name].isna(), dx_name] = 0

    if dx == 'afib':
        indata = indata.drop(['I48','C01A','C01B','B01A'],axis=1)
    if dx == 'cad':
        indata = indata.drop(['C01D','I20','I21','I22','I23','I24','I25'],axis=1)
    if dx == 'celiac':
        indata = indata.drop(['K90'],axis=1)
    if dx == 'gallstone':
        indata = indata.drop(['K80'],axis=1)
    if dx == 'polyp':
        indata = indata.drop(['J33'],axis=1)
    if dx == 't2d':
        indata = indata.drop(['E11','E14','A10B'],axis=1)
    if dx == 'varicose':
        indata = indata.drop(['I83'],axis=1)
    if dx == 'vte':
        indata = indata.drop(['I80','I81','I82','I26','B01A'],axis=1)
        
    indata = indata.dropna(thresh=100,axis=1)

    ##### Run ML
    
    condition = 'All features'

    X = indata.drop(['f.eid', dx_name],axis=1).astype(float)
    col_names = X.columns.to_list()
    X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_.\ ]+', '', x))
    cat_feat = id_cat_feat(X)
    y = indata[dx_name].astype(float)
    ids = indata['f.eid']

    metrics = []

    #####
    
    params_test = {
        'data_sample_strategy': ['goss'],
        'boosting': ['gbdt'],
        'objective': ['binary'],
        'force_col_wise': ['true'],    
        'num_iterations': [1000],
        'learning_rate': [0.1],
        'early_stopping_round': [5],
        'verbose': [0],
        'num_leaves': [20, 35, 50, 65, 80],
        'min_data_in_leaf': [50, 100, 500],
        'lambda_l2': [1, 0.1, 10, 100],
        'feature_fraction': [1, 0.9, 0.8]
    }
    keys, values = zip(*params_test.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    df = pd.DataFrame(combinations)
    param_list = df.to_dict(orient='records')

    metrics = []
    
    for params in param_list:
    
        outer_cv = KFold(n_splits=6, shuffle=True)
        for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(X)):
            X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
            y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]
            ids_test = ids.iloc[test_idx]
    
            inner_cv = KFold(n_splits=6, shuffle=True)
            for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val)):
                
                X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
                y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
                
                train_data = lgb.Dataset(X_train, label=y_train)
                val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
                # Model training
                model = lgb.train(params, train_data, valid_sets=val_data, categorical_feature=cat_feat)
                best_iter = model.best_iteration

                # Prediction & Evaluation on validation set
                y_pred_val = model.predict(X_val, num_iteration=best_iter)
                auroc_val = roc_auc_score(y_val, y_pred_val)
                auprc_val = average_precision_score(y_val, y_pred_val)
                ll_val = log_loss(y_val, y_pred_val)
        
                # Prediction & Evaluation on holdout set
                y_pred_hold = model.predict(X_test, num_iteration=best_iter)
                auroc_hold = roc_auc_score(y_test, y_pred_hold)
                auprc_hold = average_precision_score(y_test, y_pred_hold)
                ll_hold = log_loss(y_test, y_pred_hold)
                        
                # Collecting metrics
                filtered_params = {k: params[k] for k in ['num_leaves', 'min_data_in_leaf', 'lambda_l2', 'feature_fraction']}
                metrics_dict = {'Disease': dx,
                                'Outer fold': outer_fold, 'Inner fold': inner_fold,
                                'AUROC_val': auroc_val, 'AUPRC_val': auprc_val, 'LL_val': ll_val, 'Prop_val': y_val.mean(),
                                'AUROC_hold': auroc_hold, 'AUPRC_hold': auprc_hold, 'LL_hold': ll_hold, 'Prop_hold': y_test.mean()}
                metrics_dict = {**filtered_params, **metrics_dict}
                metrics.append(metrics_dict)
                print(metrics_dict)
                
    ##### Save data
        
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_pickle(f'./ML_Results/HP_Search/{dx_name}_metrics.pkl')
    metrics = 0
    metrics_df = 0

## Train ML with all features

In [ ]:
for dx in ['afib','cad','celiac','gallstone','polyp','t2d','varicose','vte']:

    dx_name = dx+'_status'

    cd = pd.read_pickle('./Final/cases.pkl')
    indata = pd.read_pickle('./Final/indata.pkl')

    indata = indata.merge(cd[['f.eid',dx_name]], on='f.eid', how='inner').drop_duplicates('f.eid')
    indata = indata.loc[indata[dx_name] != -1]
    indata.loc[indata[dx_name].isna(), dx_name] = 0

    if dx == 'afib':
        indata = indata.drop(['I48','C01A','C01B','B01A'],axis=1)
    if dx == 'cad':
        indata = indata.drop(['C01D','I20','I21','I22','I23','I24','I25'],axis=1)
    if dx == 'celiac':
        indata = indata.drop(['K90'],axis=1)
    if dx == 'gallstone':
        indata = indata.drop(['K80'],axis=1)
    if dx == 'polyp':
        indata = indata.drop(['J33'],axis=1)
    if dx == 't2d':
        indata = indata.drop(['E11','E14','A10B'],axis=1)
    if dx == 'varicose':
        indata = indata.drop(['I83'],axis=1)
    if dx == 'vte':
        indata = indata.drop(['I80','I81','I82','I26','B01A'],axis=1)
        
    indata = indata.dropna(thresh=100,axis=1)

    ##### Run ML
    
    condition = 'All features'

    X = indata.drop(['f.eid', dx_name],axis=1).astype(float)
    col_names = X.columns.to_list()
    X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_.\ ]+', '', x))
    cat_feat = id_cat_feat(X)
    y = indata[dx_name].astype(float)
    ids = indata['f.eid']

    # Placeholder for metrics and predictions
    metrics = []
    predictions = []
    importances = []
    hold_shap = []
    hold_ids = []

    # Outer 6-fold CV
    outer_cv = KFold(n_splits=6, shuffle=True)
    for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(X)):
        X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
        y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]
        ids_test = ids.iloc[test_idx]

        # Inner 6-fold CV (90% train/validation, 10% validation)
        inner_cv = KFold(n_splits=6, shuffle=True)
        for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val)):
            X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
            y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
            
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            params = {
                'data_sample_strategy': 'goss',
                'boosting': 'gbdt',
                'objective': 'binary',
                'force_col_wise': 'true',
                'num_iterations': 1000,
                'num_leaves': 50,
                'learning_rate': 0.1,
                'min_data_in_leaf': 100,
                'early_stopping_round': 5,
                'lambda_l2': 1,
                'verbose': 0,
                'feature_fraction': 1
            }
        
            # Model training
            if True:
                model = lgb.train(params, train_data, valid_sets=val_data, categorical_feature=cat_feat)
                model.save_model(f'./ML_Results/Models/{dx_name}_{condition}_{outer_fold}_{inner_fold}.model', num_iteration=model.best_iteration)
            
            if False:
                model = lgb.Booster(model_file=f'./ML_Results/Models/{dx_name}_{condition}_{outer_fold}_{inner_fold}.model')
            
            importances.append(dict(zip(col_names, model.feature_importance(importance_type='gain'))))
            if inner_fold == 0:
                hold_shap.append(model.predict(X_test, num_iteration=model.best_iteration, pred_contrib=True)[:, :-1])
                hold_ids.append(ids_test)    
            
            # Prediction & Evaluation on validation set
            y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
            auroc_val = roc_auc_score(y_val, y_pred_val)
            auprc_val = average_precision_score(y_val, y_pred_val)
    
            # Prediction & Evaluation on holdout set
            y_pred_hold = model.predict(X_test, num_iteration=model.best_iteration)
            auroc_hold = roc_auc_score(y_test, y_pred_hold)
            auprc_hold = average_precision_score(y_test, y_pred_hold)
                    
            # Collecting metrics
            metrics_dict = {'AUROC_val': auroc_val, 'AUPRC_val': auprc_val,
                            'AUROC_hold': auroc_hold, 'AUPRC_hold': auprc_hold}
            metrics.append(metrics_dict)
            print(metrics_dict)
            
            # Collecting predictions with ids and fold info
            predictions.extend(zip(ids_test, y_pred_hold))
            
            print(outer_fold, inner_fold)
            
    ##### Save data
    
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_pickle(f'./ML_Results/Metrics/{dx_name}_{condition}_metrics.pkl')
    metrics = 0
    metrics_df = 0

    imp_df = pd.DataFrame(importances)
    imp_df.to_pickle(f'./ML_Results/Importance/{dx_name}_{condition}_gain.pkl')
    #imp_df.mean().reset_index().to_excel('fi.xlsx')
    importances = 0
    imp_df = 0

    predictions_df = pd.DataFrame(predictions, columns=['f.eid', 'Prediction']).groupby('f.eid')['Prediction'].mean().reset_index()
    predictions_df = predictions_df.merge(indata[['f.eid',dx_name]])
    predictions_df.to_pickle(f'./ML_Results/Predictions/{dx_name}_{condition}_hold_predictions.pkl')
    predictions = 0
    predictions_df = 0

    dfs = []

    for shap_array, ids in zip(hold_shap, hold_ids):
        df = pd.DataFrame(shap_array, columns=col_names)
        df['f.eid'] = np.array(ids).flatten()
        dfs.append(df)

    dfs = pd.concat(dfs, ignore_index=True)
    dfs = dfs.groupby('f.eid').mean().reset_index()
    dfs.to_pickle(f'./ML_Results/Importance/{dx_name}_{condition}_shap.pkl')
    hold_shap = 0
    hold_ids = 0
    dfs = 0

    train_data = 0
    val_data = 0
    X_train_val = 0
    X_train = 0
    X_val = 0
    X_test = 0

## Train ML with MR-selected features

In [ ]:
for dx in ['afib','cad','celiac','gallstone','polyp','t2d','varicose','vte']:

    dx_name = dx+'_status'

    cd = pd.read_pickle('./Final/cases.pkl')
    cond = pd.read_pickle('./MR_Results/mr_conditions.pkl')
    all_feat = ['f.eid','sex','age','Fasting time'] + list(cond.loc[(cond['Disease'] == dx) & (cond['Condition'] == 'All MR tested')]['Features'].iloc[0])
    indata = pd.read_pickle('./Final/indata.pkl')[all_feat]

    indata = indata.merge(cd[['f.eid',dx_name]], on='f.eid', how='inner').drop_duplicates('f.eid')
    indata = indata.loc[indata[dx_name] != -1]
    indata.loc[indata[dx_name].isna(), dx_name] = 0

    if dx == 'afib':
        indata = indata.drop(['I48','C01A','C01B','B01A'],axis=1,errors='ignore')
    elif dx == 'cad':
        indata = indata.drop(['C01D','I20','I21','I22','I23','I24','I25'],axis=1,errors='ignore')
    elif dx == 'celiac':
        indata = indata.drop(['K90'],axis=1,errors='ignore')
    elif dx == 'gallstone':
        indata = indata.drop(['K80'],axis=1,errors='ignore')
    elif dx == 'polyp':
        indata = indata.drop(['J33'],axis=1,errors='ignore')
    elif dx == 't2d':
        indata = indata.drop(['E11','E14','A10B'],axis=1,errors='ignore')
    elif dx == 'varicose':
        indata = indata.drop(['I83'],axis=1,errors='ignore')
    elif dx == 'vte':
        indata = indata.drop(['I80','I81','I82','I26','B01A'],axis=1,errors='ignore')
        
    indata = indata.dropna(thresh=100,axis=1)

    ##### Run ML
    
    for condition in cond['Condition'].unique():
        print(condition)

        feat_inc = ['age', 'sex', 'Fasting time'] + list(cond.loc[(cond['Disease'] == dx) & (cond['Condition'] == condition)]['Features'].iloc[0])
        feat_inc = [col for col in feat_inc if col in indata.columns]

        X = indata.drop(['f.eid', dx_name],axis=1)[feat_inc].astype(float)
        col_names = X.columns.to_list()
        X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_.\ ]+', '', x))
        cat_feat = id_cat_feat(X)
        y = indata[dx_name].astype(float)
        ids = indata['f.eid']

        # Placeholder for metrics and predictions
        metrics = []
        predictions = []
        importances = []
        hold_shap = []
        hold_ids = []

        # Outer 6-fold CV
        outer_cv = KFold(n_splits=6, shuffle=True)
        for outer_fold, (train_val_idx, test_idx) in enumerate(outer_cv.split(X)):
            X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
            y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]
            ids_test = ids.iloc[test_idx]

            # Inner 6-fold CV (90% train/validation, 10% validation)
            inner_cv = KFold(n_splits=6, shuffle=True, random_state=42)
            for inner_fold, (train_idx, val_idx) in enumerate(inner_cv.split(X_train_val)):
                X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
                y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
                
                train_data = lgb.Dataset(X_train, label=y_train)
                val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
                
                params = {
                    'data_sample_strategy': 'goss',
                    'boosting': 'gbdt',
                    'objective': 'binary',
                    'force_col_wise': 'true',
                    'num_iterations': 1000,
                    'num_leaves': 50,
                    'learning_rate': 0.1,
                    'min_data_in_leaf': 100,
                    'early_stopping_round': 5,
                    'lambda_l2': 1,
                    'verbose': 0,
                    'feature_fraction': 1
                }
            
                # Model training
                if True:
                    model = lgb.train(params, train_data, valid_sets=val_data, categorical_feature=cat_feat)
                    model.save_model(f'./ML_Results/Models/{dx_name}_{condition}_{outer_fold}_{inner_fold}.model', num_iteration=model.best_iteration)
                
                if False:
                    model = lgb.Booster(model_file=f'./ML_Results/Models/{dx_name}_{condition}_{outer_fold}_{inner_fold}.model')
                
                importances.append(dict(zip(col_names, model.feature_importance(importance_type='gain'))))
                if inner_fold == 0:
                    hold_shap.append(model.predict(X_test, num_iteration=model.best_iteration, pred_contrib=True)[:, :-1])
                    hold_ids.append(ids_test)    
                
                # Prediction & Evaluation on validation set
                y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
                auroc_val = roc_auc_score(y_val, y_pred_val)
                auprc_val = average_precision_score(y_val, y_pred_val)
        
                # Prediction & Evaluation on holdout set
                y_pred_hold = model.predict(X_test, num_iteration=model.best_iteration)
                auroc_hold = roc_auc_score(y_test, y_pred_hold)
                auprc_hold = average_precision_score(y_test, y_pred_hold)
                        
                # Collecting metrics
                metrics_dict = {'AUROC_val': auroc_val, 'AUPRC_val': auprc_val,
                                'AUROC_hold': auroc_hold, 'AUPRC_hold': auprc_hold}
                metrics.append(metrics_dict)
                print(metrics_dict)
                
                # Collecting predictions with ids and fold info
                predictions.extend(zip(ids_test, y_pred_hold))
                
                print(outer_fold, inner_fold)
                
        ##### Save data
        
        metrics_df = pd.DataFrame(metrics)
        metrics_df.to_pickle(f'./ML_Results/Metrics/{dx_name}_{condition}_metrics.pkl')
        metrics = 0
        metrics_df = 0

        imp_df = pd.DataFrame(importances)
        imp_df.to_pickle(f'./ML_Results/Importance/{dx_name}_{condition}_gain.pkl')
        #imp_df.mean().reset_index().to_excel('fi.xlsx')
        importances = 0
        imp_df = 0

        predictions_df = pd.DataFrame(predictions, columns=['f.eid', 'Prediction']).groupby('f.eid')['Prediction'].mean().reset_index()
        predictions_df = predictions_df.merge(indata[['f.eid',dx_name]])
        predictions_df.to_pickle(f'./ML_Results/Predictions/{dx_name}_{condition}_hold_predictions.pkl')
        predictions = 0
        predictions_df = 0

        dfs = []

        for shap_array, ids in zip(hold_shap, hold_ids):
            df = pd.DataFrame(shap_array, columns=col_names)
            df['f.eid'] = np.array(ids).flatten()
            dfs.append(df)

        dfs = pd.concat(dfs, ignore_index=True)
        dfs = dfs.groupby('f.eid').mean().reset_index()
        dfs.to_pickle(f'./ML_Results/Importance/{dx_name}_{condition}_shap.pkl')
        hold_shap = 0
        hold_ids = 0
        dfs = 0

        train_data = 0
        val_data = 0
        X_train_val = 0
        X_train = 0
        X_val = 0
        X_test = 0

## Bootstrap metrics

In [ ]:
conditions = ['All features','All MR tested',
              'Any p < 0.05','No p < 0.05',
              'Any p < 0.01', 'Any p < 0.001', 'Any Bonferroni',
              'IVW p < 0.05', 'IVW p < 0.01', 'IVW p < 0.001', 'IVW Bonferroni',
              'IVW p < 0.05 + 3 robust', 'IVW p < 0.01 + 3 robust','IVW p < 0.001 + 3 robust', 'IVW Bonferroni + 3 robust']

for dx in ['afib','cad','celiac','gallstone','polyp','t2d','varicose','vte']:
    dx_name = dx+'_status'
    for cond in conditions:
        hp = pd.read_pickle(f'/Users/robchiral/Desktop/MR_ML/ML_Results/Predictions/{dx_name}_{cond}_hold_predictions.pkl')
        auroc = bootstrap(hp, 'Prediction', dx_name, roc_auc_score, n_bootstrap=500)
        auprc = bootstrap(hp, 'Prediction', dx_name, average_precision_score, n_bootstrap=500)
        brier = bootstrap(hp, 'Prediction', dx_name, brier_score_loss, n_bootstrap=500)
        logloss = bootstrap(hp, 'Prediction', dx_name, log_loss, n_bootstrap=500)
        
        bs_metrics = pd.DataFrame([{
            'Outcome': dx,
            'Condition': cond,
            'AUROC': auroc[0], 'AUROC_5': auroc[1], 'AUROC_95': auroc[2],
            'AUPRC': auprc[0], 'AUPRC_5': auprc[1], 'AUPRC_95': auprc[2],
            'Brier': brier[0], 'Brier_5': brier[1], 'Brier_95': brier[2],
            'LogLoss': logloss[0], 'LogLoss_5': logloss[1], 'LogLoss_95': logloss[2]
        }])
        
        bs_metrics.to_csv(f'./ML_Results/Bootstrap/Individual/{dx_name}_{cond}_bs_metrics.csv', index=False)